# import



In [1]:
ls -lt /jukebox/norman/rmasis/MemPal/analysis/MemPal2024/PythonData2024/Logs/reliability2feature/ | head -n 5

total 762016
-rw-rw-r--. 1 rmasis normanlab  749 Apr 28 16:27 slurm_-848382_1363.log
-rw-rw-r--. 1 rmasis normanlab  749 Apr 28 16:27 slurm_-848381_1362.log
-rw-rw-r--. 1 rmasis normanlab  749 Apr 28 16:26 slurm_-848414_1395.log
-rw-rw-r--. 1 rmasis normanlab  749 Apr 28 16:26 slurm_-848434_1415.log
ls: write error: Broken pipe


In [2]:
from tqdm import tqdm_notebook
import deepdish as dd
import numpy as np
import os
import h5py
import pandas as pd
from scipy import stats
from _searchlight_util import *
import matplotlib.pyplot as plt
from _classification_util import Convert_SourceItem_to_PairedItem_Order
from _mempal_util import room_obj_assignments, save_obj, load_obj, create_dirs, subject_ids

trials = ['GR0', 'GR1', 'GR2', 'GR3','GR4', 'GR5', 'GR6', 'GR7','GR8', 'GR9', 'GR10','FR']


In [5]:
roi = 'SL'
hems=['L','R']
nPerm = 1000
nSubj =25
nRooms = 23
nItems=23
nTrials = 23

scores = {}
rois = []


# date = 20240108


measure_list = [
#     'betas', # contains property list from regrssion
    'r_squared', # from regression
    'intercepts', # from regression
#     'occupied_box_volume', 
#     'boxcoll_volume', 
    'ratio_occupied_boxcoll_volume', 
    "area", 
    "manual_object_count", 
    "num_corners",
    "view_outside",
    "degree",
#     'betweenness',
#     'closeness',
#     'eigenvector',
#     'pagerank',
    
    ]

property_list = [
    'occupied_box_volume', 
    'boxcoll_volume', 
    'ratio_occupied_boxcoll_volume', 
    "area", 
    "manual_object_count", 
    "num_corners",
    "view_outside",
    "degree",
    'betweenness',
#     'closeness',
    'eigenvector',
    'pagerank'
    ]

# 202504232 
property_list = [
#     'occupied_box_volume', 
#     'boxcoll_volume', 
    'ratio_occupied_boxcoll_volume', 
    "area", 
    "manual_object_count", 
    "num_corners",
    "view_outside",
    "degree",
    
#     'betweenness',
#     'closeness',
#     'eigenvector',
#     'pagerank'
    ]


date = 20250417 # 
date = 20250423 # including regression
date = 202504232
date = 20250425 # permutation regression

date = 20250524 # revision update

# scores = {}
# for measure in tqdm_notebook(measure_list):
#     scores[measure] = {}
#     for trial_type in ['GR','FR']:
#         scores[measure] = {}

#         for hem in ['L','R']:

#             scores[measure][hem] = np.zeros((len(SLlist[hem]),nSubj, nPerm+1))

#             for roi_id in (range(len(SLlist[hem]))[:]):
   
#                 dirpath = '../PythonData2024/Output/Reliability2RoomFeatures'
#                 fname = '{}_{}{:03d}_{}_reliability2roomfeatures'.format(date,roi,roi_id,hem) + '.h5'

#                 fullpath = os.path.join(dirpath,fname)
#                 scores[measure][hem][roi_id] =  dd.io.load(fullpath,'/{}'.format(measure))



scores = {}             
for measure in tqdm_notebook(measure_list):
    scores[measure] = {}

    for hem in ['L','R']:
        
        if measure not in ['betas','r_squared','intercepts']:
            scores[measure][hem] = np.zeros((len(SLlist[hem]),nSubj, nPerm+1))
            
        elif measure in ['r_squared' ,'intercepts']:
#             scores[measure][hem]  = np.zeros((len(SLlist[hem]),nSubj))
            scores[measure][hem]  = np.zeros((len(SLlist[hem]),nSubj, nPerm+1))
            

        ### populates correlation reasults (spearman corr)
        ### and populates betas for intercepts from multiple regression
        ### and populates r2 for multiple regresion
        for roi_id in (range(len(SLlist[hem]))[:]):

            dirpath = '../PythonData2024/Output/Reliability2RoomFeatures'
            fname = '{}_{}{:03d}_{}_reliability2roomfeatures'.format(date,roi,roi_id,hem) + '.h5'

            fullpath = os.path.join(dirpath,fname)
            scores[measure][hem][roi_id] =  dd.io.load(fullpath,'/{}'.format(measure))


### REGRESSION STUFF
scores['betas'] = {}
for measure in tqdm_notebook(property_list):
    scores['betas'][measure] = {}
    for hem in ['L','R']:
#         scores['betas'][measure][hem] = np.zeros((len(SLlist[hem]),nSubj))
        scores['betas'][measure][hem] = np.zeros((len(SLlist[hem]),nSubj,nPerm+1))

        for roi_id in (range(len(SLlist[hem]))[:]):
            dirpath = '../PythonData2024/Output/Reliability2RoomFeatures'
            fname = '{}_{}{:03d}_{}_reliability2roomfeatures'.format(date,roi,roi_id,hem) + '.h5'

            fullpath = os.path.join(dirpath,fname)
            scores['betas'][measure][hem][roi_id] =  dd.io.load(fullpath,'/{}/{}'.format('betas',measure))





/jukebox/pkgs/PYGER/base/envs/0.11.0/lib/python3.7/site-packages/ipykernel_launcher.py:96: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/8 [00:00<?, ?it/s]

/jukebox/pkgs/PYGER/base/envs/0.11.0/lib/python3.7/site-packages/ipykernel_launcher.py:123: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/6 [00:00<?, ?it/s]

In [6]:
###
### ROCN and RRCN x2 control
###

top_amount = 2 ;

# top_r2r_ev = {} # reliabilty ~ evidence
for measure in measure_list:
#     top_r2r_ev[network]  = {}
    
    if measure not in ['betas','r_squared','intercepts']:
        print("....* {} *....".format(measure))
        for hem in ['L','R']:
            subj_mean = scores[measure][hem][:,:,0].mean(1) #average subj
            high_corr_idx = np.argsort(subj_mean)[::-1][:top_amount]

            print(high_corr_idx)
            print('corrs: ', measure, hem, subj_mean[high_corr_idx].mean())

    #             top_r2r_ev[network][hem] = high_corr_idx



....* ratio_occupied_boxcoll_volume *....
[ 22 539]
corrs:  ratio_occupied_boxcoll_volume L 0.12379446640316208
[1158  127]
corrs:  ratio_occupied_boxcoll_volume R 0.1757905138339921
....* area *....
[ 633 1250]
corrs:  area L 0.07855731225296443
[575 501]
corrs:  area R 0.08687747035573123
....* manual_object_count *....
[158 895]
corrs:  manual_object_count L 0.12583439801949142
[314  37]
corrs:  manual_object_count R 0.141478407266394
....* num_corners *....
[145 917]
corrs:  num_corners L 0.16949588415766276
[409 613]
corrs:  num_corners R 0.1709338020553406
....* view_outside *....
[903 902]
corrs:  view_outside L 0.12402374011409312
[172 444]
corrs:  view_outside R 0.15258357792288213
....* degree *....
[ 379 1394]
corrs:  degree L 0.10053433127427802
[ 571 1004]
corrs:  degree R 0.12223362594921997


In [8]:
import scipy.stats as stats 



nv = 40962 #verts in fsaverage6 brain
reg_list = [
    'occupied_box_volume_reg', 
    'boxcoll_volume_reg', 
    'ratio_occupied_boxcoll_volume_reg', 
    "area_reg", 
    "manual_object_count_reg", 
    "num_corners_reg",
    "view_outside_reg",
    "degree_reg",
    'betweenness_reg',
#     'closeness',
    'eigenvector_reg',
    'pagerank_reg'
    ]


# 202504232 
reg_list = [
#     'occupied_box_volume', 
#     'boxcoll_volume', 
    'ratio_occupied_boxcoll_volume_regperm',
    'ratio_occupied_boxcoll_volume_reg', 
    "area_reg", 
    "manual_object_count_reg", 
    "num_corners_reg",
    "view_outside_reg",
    "degree_reg",
    
    'ratio_occupied_boxcoll_volume_regperm', 
    "area_regperm", 
    "manual_object_count_regperm", 
    "num_corners_regperm",
    "view_outside_regperm",
    "degree_regperm",
#     'betweenness',
#     'closeness',
#     'eigenvector',
#     'pagerank'
    ]


# ### CORRELATION RESULTS

results = {}
for measuretype in tqdm_notebook(measure_list):
    
    if measuretype not in ['betas','r_squared','intercepts']:
        print(measuretype)
        results[measuretype] = {}

        #average across subj from (nSLs,nSubj,nPerm+1) to (nSLs,nPerm+1)
        ## average out subjects
        dd_hem =  {'L':scores[measuretype]['L'].mean(1),
                   'R':scores[measuretype]['R'].mean(1)} 

        dd_vox = SLtoVox(dd_hem, ROIlist,nv,zeronan=False) #

        raw = {'L':[],'R':[]}
        z = {'L':[],'R':[]}
        for hem in ['L','R']:
            z[hem] = nullZ(dd_vox[hem])
            raw[hem] = dd_vox[hem][:,0] #grab the true val, the rest are perms

    #             q = FDR_z_hem(z)

        results[measuretype]['raw'] = raw
        results[measuretype]['z'] = z
        results[measuretype]['q'] = FDR_z_hem(z,sided=2)


###### REGRESSION RESULTS

for measuretype in tqdm_notebook(reg_list):

    if not 'regperm' in measuretype: # for just the 'reg'
        t_vox = {}
        t_p_vox = {}
        raw_vox = {}
        
        print(measuretype)
        
        results[measuretype] = {}

        #keep subject dimension but only grap true observation (not perms)
        # nSLs, nsubj, nperm --> nsls,nsubj
        dd_hem =  {'L':scores['betas'][measuretype.split("_reg")[0]]['L'][:,:,0],
                   'R':scores['betas'][measuretype.split("_reg")[0]]['R'][:,:,0]}
        

        dd_vox = SLtoVox(dd_hem, ROIlist,nv,zeronan=False) #

        for hem in ['L','R']:
            t_vox[hem] = np.full((nv),fill_value=np.nan)
            t_p_vox[hem] = np.full((nv),fill_value=np.nan)
            raw_vox[hem] = np.nanmean(dd_vox[hem],1) #avg out subj
            for vi in tqdm_notebook(range(nv)):

                # get t value from 25 subj numbers for every vertex
#                     t, p = stats.ttest_1samp(dd_vox[hem][vi,:],chance,alternative='greater') #one sided t-test 
                t, p = stats.ttest_1samp(dd_vox[hem][vi],0) #only two-sided available
                p = p # / 2 # make it 1 sided
                t_vox[hem][vi] = t
                t_p_vox[hem][vi] = p

        results[measuretype]['t_q'] = FDR_p_hem(t_p_vox)
        results[measuretype]['t']   = t_vox
        results[measuretype]['raw'] = raw_vox
        
    elif 'regperm' in measuretype:
        print(measuretype)
        results[measuretype] = {}

        # avg out subj to keep nPerm+1
        dd_hem =  {'L':np.nanmean(scores['betas'][measuretype.split("_regperm")[0]]['L'],1),
                   'R':np.nanmean(scores['betas'][measuretype.split("_regperm")[0]]['R'],1)}
        

        dd_vox = SLtoVox(dd_hem, ROIlist,nv,zeronan=False) #
        
        raw = {'L':[],'R':[]}
        z = {'L':[],'R':[]}
        for hem in ['L','R']:
            z[hem] = nullZ(dd_vox[hem])
            raw[hem] = dd_vox[hem][:,0] #grab the true val, the rest are perms

    #             q = FDR_z_hem(z)

        results[measuretype]['raw'] = raw
        results[measuretype]['z'] = z
        results[measuretype]['q'] = FDR_z_hem(z,sided=2)
        
        

/jukebox/pkgs/PYGER/base/envs/0.11.0/lib/python3.7/site-packages/ipykernel_launcher.py:50: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/8 [00:00<?, ?it/s]

ratio_occupied_boxcoll_volume
area
manual_object_count
num_corners
view_outside
degree


/jukebox/pkgs/PYGER/base/envs/0.11.0/lib/python3.7/site-packages/ipykernel_launcher.py:78: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/13 [00:00<?, ?it/s]

...

  0%|          | 0/40962 [00:00<?, ?it/s]

degree_reg


  0%|          | 0/40962 [00:00<?, ?it/s]

  0%|          | 0/40962 [00:00<?, ?it/s]

ratio_occupied_boxcoll_volume_regperm
area_regperm
manual_object_count_regperm
num_corners_regperm
view_outside_regperm
degree_regperm


In [9]:
from _mempal_util import save_obj

### DATA
data = results




savedate = 20250417
savedate = 20250423 #includes regression coeff results
savedate = 202504232
savedate = 20250425 # permutation regressions

savedate = 20250524 # revision update
fname2save = '../PythonData2024/Output/brainmaps/{}_{}2efeatures.pkl'.format(savedate,
                                                                                   'reliability')


save_obj(fname2save,data)

print('...', fname2save)

print("complete")

# print('../../PythonData/MemPal2021/output/brainmaps/{}_SL_room2room_cp.pkl'.format(date))



... ../PythonData2024/Output/brainmaps/20250524_reliability2efeatures.pkl
complete
